In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0),  
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2, padding=0),
            nn.ELU(), 
            nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dim_feedforward=None, activation=nn.ELU):
        super().__init__()
        if dim_feedforward is None:
            dim_feedforward = 4 * d_model
        self.self_attn = nn.MultiheadAttention(d_model, num_heads, dropout=0.0, batch_first=True)
        # Implementation of feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.activation = activation()
        
    def forward(self, query, key, value):
        src = query
        src2 = self.self_attn(query=query, key=key, value=value)[0]
        src = src + src2
        src2 = self.linear2(self.activation(self.linear1(src2)))
        src = src + src2
        return src

In [4]:
class FusionNet(nn.Module):
    def __init__(self, d_model=8, num_heads=1):
        super().__init__()
        self.img_preproc = nn.Sequential(
            nn.Linear(512, 2048),
            nn.ELU(),
        )
        
        self.target_cross_attn_1 = TransformerEncoderLayer(d_model=d_model, num_heads=num_heads)
        self.img_cross_attn_1 = TransformerEncoderLayer(d_model=d_model, num_heads=num_heads)
        self.conv_1 = nn.Conv3d(d_model, 2 * d_model, kernel_size=3, stride=1)
        self.act_1 = nn.ELU()
        
        self.target_cross_attn_2 = TransformerEncoderLayer(d_model=2 * d_model, num_heads=num_heads)
        self.img_cross_attn_2 = TransformerEncoderLayer(d_model=2 * d_model, num_heads=num_heads)
        self.conv_2 = nn.Conv3d(2 * d_model, 4 * d_model, kernel_size=3, stride=1)
        self.act_2 = nn.ELU()
        
        self.target_cross_attn_3 = TransformerEncoderLayer(d_model=4 * d_model, num_heads=num_heads)
        self.img_cross_attn_3 = TransformerEncoderLayer(d_model=4 * d_model, num_heads=num_heads)
        self.conv_3 = nn.Conv3d(4 * d_model, 8 * d_model, kernel_size=3, stride=1)
        self.act_3 = nn.ELU()
        
        self.target_cross_attn_4 = TransformerEncoderLayer(d_model=8 * d_model, num_heads=num_heads)
        self.img_cross_attn_4 = TransformerEncoderLayer(d_model=8 * d_model, num_heads=num_heads)
        self.conv_4 = nn.Conv3d(8 * d_model, 16 * d_model, kernel_size=3, stride=1)
        self.act_4 = nn.ELU()
        
        self.max_pool = nn.MaxPool3d(kernel_size=(1, 3, 3))
        
        
    def forward(self, target, img_features):
        batch_size = target.shape[0]
        
        img_features = self.img_preproc(img_features)
        
        # layer 1
        target = target.permute(0, 2, 3, 4, 1).reshape(batch_size, 9*11*11, 8)
        img = img_features.reshape(batch_size, 256, 8)
        target_1 = self.target_cross_attn_1(query=target, key=img, value=img)
        img_1 = self.img_cross_attn_1(query=img, key=target, value=target)
        target_1 = target_1.reshape(batch_size, 9, 11, 11, 8).permute(0, 4, 1, 2, 3)
        target_1 = self.act_1(self.conv_1(target_1))
        img_1 = img_1.reshape(batch_size, 2048)
        
        # layer 2
        target_1 = target_1.permute(0, 2, 3, 4, 1).reshape(batch_size, 7*9*9, 16)
        img_1 = img_1.reshape(batch_size, 128, 16)
        target_2 = self.target_cross_attn_2(query=target_1, key=img_1, value=img_1)
        img_2 = self.img_cross_attn_2(query=img_1, key=target_1, value=target_1)
        target_2 = target_2.reshape(batch_size, 7, 9, 9, 16).permute(0, 4, 1, 2, 3)
        target_2 = self.act_2(self.conv_2(target_2))
        img_2 = img_2.reshape(batch_size, 2048)
        
        # layer 3
        target_2 = target_2.permute(0, 2, 3, 4, 1).reshape(batch_size, 5*7*7, 32)
        img_2 = img_2.reshape(batch_size, 64, 32)
        target_3 = self.target_cross_attn_3(query=target_2, key=img_2, value=img_2)
        img_3 = self.img_cross_attn_3(query=img_2, key=target_2, value=target_2)
        target_3 = target_3.reshape(batch_size, 5, 7, 7, 32).permute(0, 4, 1, 2, 3)
        target_3 = self.act_3(self.conv_3(target_3))
        img_3 = img_3.reshape(batch_size, 2048)
        
        # layer 4
        target_3 = target_3.permute(0, 2, 3, 4, 1).reshape(batch_size, 3*5*5, 64)
        img_3 = img_3.reshape(batch_size, 32, 64)
        target_4 = self.target_cross_attn_4(query=target_3, key=img_3, value=img_3)
        img_4 = self.img_cross_attn_4(query=img_3, key=target_3, value=target_3)
        target_4 = target_4.reshape(batch_size, 3, 5, 5, 64).permute(0, 4, 1, 2, 3)
        target_4 = self.act_4(self.conv_4(target_4))
        img_4 = img_4.reshape(batch_size, 2048)
        
        
        target_4 = self.max_pool(target_4)
        
        features = target_4.reshape(batch_size, -1)
        
        return features

In [5]:
from torch.nn.functional import one_hot

class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        visual_features_dim = 512
        target_features_dim = 9 * 11 * 11 
        self.visual_encoder = VisualEncoder()
        self.visual_encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AngelaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.target_encoder = nn.Sequential(
            nn.Conv3d(7, 8, kernel_size=1, stride=1, padding=0),
            nn.ELU(),
        )
        policy_hidden_dim = 128 
        self.policy_network = FusionNet()
        
        self.action_head = nn.Linear(policy_hidden_dim, action_space.n)
        self.value_head = nn.Linear(policy_hidden_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.visual_encoder.cuda()
            self.target_encoder.cuda()
            self.policy_network.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs']
        pov = obs['pov'].permute(0, 3, 1, 2).float() / 255.0
        target = one_hot(obs['target_grid'].long(), num_classes=7).permute(0, 4, 1, 2, 3).float()
        if self.use_cuda:
            pov.cuda()
            target.cuda()
            
        with torch.no_grad():
            visual_features = self.visual_encoder(pov)
            
        target_features = self.target_encoder(target)
        
        features = self.policy_network(target_features, visual_features)
        
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [6]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [7]:
class VisualObservationWrapper(ObsWrapper):
    def __init__(self, env, include_target=False):
        super().__init__(env)
        self.observation_space = {   
            'pov': gym.spaces.Box(low=0, high=255, shape=(64, 64, 3)),
            'inventory': gym.spaces.Box(low=0.0, high=20.0, shape=(6,)),
            'compass': gym.spaces.Box(low=-180.0, high=180.0, shape=(1,))
        }
        if include_target:
            self.observation_space['target_grid'] = \
                gym.spaces.Box(low=0, high=6, shape=(9, 11, 11))
        self.observation_space = gym.spaces.Dict(self.observation_space)

    def observation(self, obs, reward=None, done=None, info=None):
        if info is not None:
            if 'target_grid' in info:
                target_grid = info['target_grid']
                del info['target_grid']
            else:
                logger.error(f'info: {info}')
                if hasattr(self.unwrapped, 'should_reset'):
                    self.unwrapped.should_reset(True)
                target_grid = self.env.unwrapped.tasks.current.target_grid
        else:
            target_grid = self.env.unwrapped.tasks.current.target_grid
        return {
            'pov': obs['pov'].astype(np.float32),
            'inventory': obs['inventory'],
            'compass': np.array([obs['compass']['angle'].item()]),
            'target_grid': target_grid
        }

In [8]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

tasks = []
for i in range(1,156):
    if ('C'+str(i)) == 'C38': continue
    tasks.append('C'+str(i))
    
class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        if abs(rew) == 1:
            rew /= 10
            
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=250)
    env.update_taskset(TaskSet(preset=tasks))
    #env = PovOnlyWrapper(env)
    env = VisualObservationWrapper(env, include_target=True)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 3,
             "sgd_minibatch_size": 128,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 5_000,
             #"lr": 1e-4,
             #"gamma": 0.99,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO All Tasks pretrained (visual pretrained AngelaCNN + CrossAttn) (3 noops after placement) r: -0.01 div10"
                  }
              }

        },
        loggers=[WandbLogger],
        local_dir="/IGLU-Minecraft/checkpoints/all_tasks_cross_attn",
        keep_checkpoints_num=50,
        checkpoint_freq=5,
        checkpoint_at_end=True)

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-11-14 14:38:42,674	INFO wandb.py:170 -- Already logged into W&B.
2021-11-14 14:38:42,686	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_my_env_907c1_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id 04314_00000 but id 907c1_00000 is set.
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=154354) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=154354)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=154354) 2021-11-14 14:38:46,158	WARNING ppo.py:143 -- `train_batch_size` (5000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1666.
(pid=154354) 2021-11-14 14:38:46,158	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=154354) 2021-11-14 14:38:46,158	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=154353) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=154353)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=154349) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by

Result for PPO_my_env_907c1_00000:
  agent_timesteps_total: 9996
  custom_metrics: {}
  date: 2021-11-14_14-45-00
  done: false
  episode_len_mean: 99.03030303030303
  episode_media: {}
  episode_reward_max: 4.700000000000003
  episode_reward_mean: -0.5804040404040409
  episode_reward_min: -1.450000000000001
  episodes_this_iter: 99
  episodes_total: 99
  experiment_id: 214763e727544e648f411667af87eede
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999998
          cur_lr: 5.000000000000001e-05
          entropy: 2.8841074453459847
          entropy_coeff: 0.01
          kl: 0.004704763754277773
          policy_loss: -0.011509279726853228
          total_loss: 0.040411061462030835
          vf_explained_var: -0.3121291697025299
          vf_loss: 0.07982046209319503
    num_agent_steps_sampled: 9996
    num_agent_steps_trained: 9996
    num_steps_sampled: 9996
    num_steps_trained: 9996
  iteratio

...

(pid=154353) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=154353)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=154352) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=154352)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for PPO_my_env_907c1_00000:
  agent_timesteps_total: 589764
  custom_metrics: {}
  date: 2021-11-14_19-35-11
  done: false
  episode_len_mean: 97.36893203883496
  episode_media: {}
  episode_reward_max: 9.760000000000002
  episode_reward_mean: 2.061650485436899
  episode_reward_min: -1.7300000000000004
  episodes_this_iter: 103
  episodes_total: 6116
  experiment_id: 214763e727544e648f411667af87eede
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 2.562890625
          cur_lr: 5.000000000000001e-05
          entropy: 2.5746788846121893
          entropy_coeff: 0.01
          kl: 0.016106837603738735
          policy_loss: -0.0678455212006234
          total_loss: 0.12461793024140673
          vf_explained_var: 0.7892800569534302
          vf_loss: 0.17693017796796356
    num_agent_steps_sampled: 589764
    num_agent_steps_trained: 589764
    num_steps_sampled: 589764
    num_steps_trained: 589764
  iteratio

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_907c1_00000,RUNNING,192.168.3.5:154354,59,17775.1,589764,2.06165,9.76,-1.73,97.3689


Result for PPO_my_env_907c1_00000:
  agent_timesteps_total: 599760
  custom_metrics: {}
  date: 2021-11-14_19-40-04
  done: false
  episode_len_mean: 99.27450980392157
  episode_media: {}
  episode_reward_max: 10.34000000000002
  episode_reward_mean: 2.465294117647065
  episode_reward_min: -2.0599999999999987
  episodes_this_iter: 102
  episodes_total: 6218
  experiment_id: 214763e727544e648f411667af87eede
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 2.562890625
          cur_lr: 5.000000000000001e-05
          entropy: 2.577608354275043
          entropy_coeff: 0.01
          kl: 0.015623231323259202
          policy_loss: -0.06602522920833057
          total_loss: 0.08886749180328324
          vf_explained_var: 0.8619842529296875
          vf_loss: 0.14062817245522816
    num_agent_steps_sampled: 599760
    num_agent_steps_trained: 599760
    num_steps_sampled: 599760
    num_steps_trained: 599760
  iteratio

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_907c1_00000,RUNNING,192.168.3.5:154354,60,18068.2,599760,2.46529,10.34,-2.06,99.2745
